In [266]:
import numpy as np
import pandas as pd

In [268]:
data=pd.read_csv("titanic_train.csv")

In [270]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [272]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [274]:
columns_to_drop=["PassengerId","Cabin","Name","Embarked","Ticket"]

In [276]:
data_clean=data.drop(columns_to_drop,axis=1)

In [278]:
data_clean

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000
887,1,1,female,19.0,0,0,30.0000
888,0,3,female,NaN,1,2,23.4500
889,1,1,male,26.0,0,0,30.0000


In [280]:
data_clean=data_clean.fillna(data_clean["Age"].mean())

In [282]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int64(4), object(1)
memory usage: 48.9+ KB


In [284]:
X=data_clean[["Pclass","Sex","Age","SibSp","Parch","Fare"]]
Y=data_clean["Survived"]

In [286]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data_clean["Sex"]=le.fit_transform(data_clean["Sex"])

## Calculating Entropy

In [288]:
def entropy(col):
    counts=np.unique(col,return_counts=True)
    N=float(col.shape[0])
    entropy=0.0
    for i in counts[1]:
        p=i/N
        entropy+=(-1.0*np.log2(p)*p)
    return entropy

## Calculating Information Gain

In [290]:
def divide_data(x_data,fkey,fval):
    x_right=pd.DataFrame([],columns=x_data.columns)
    x_left=pd.DataFrame([],columns=x_data.columns)
    for i in range(x_data.shape[0]):
        val=x_data[fkey].loc[i]
        if val>fval:
            x_right=x_right.append(x_data.loc[i])
        else:
            x_left=x_left.append(x_data.loc[i])
    return x_right,x_left

In [292]:
def information_gain(x_data,fkey,fval):
    left,right=divide_data(x_data,fkey,fval)
    l=float(left.shape[0])/x_data.shape[0]
    r=float(right.shape[0])/x_data.shape[0]
    if left.shape[0]==0 or right.shape[0]==0:
        return -1000000 #Min Information Gain
    i_gain=entropy(x_data.Survived)-(l*entropy(left.Survived)+r*entropy(right.Survived))
    return i_gain

## Decision Tree Class

In [295]:
class DecisionTree:
    #Constructor
    def __init__(self,depth=0,max_depth=5):
        self.left=None
        self.right=None
        self.fkey=None
        self.fval=None
        self.max_depth=max_depth
        self.depth=depth
        self.target=None
    def train(self,x_train):
        features=["Pclass","Sex","Age","SibSp","Parch","Fare"]
        info_gains=[]
        for ix in features:
            i_gain=information_gain(x_train,ix,x_train[ix].mean())
            info_gains.append(i_gain)
        self.fkey=features[np.argmax(info_gains)]
        self.fval=x_train[self.fkey].mean()
        
        
        #Split data
        data_left,data_right=divide_data(x_train,self.fkey,self.fval)
        data_left=data_left.reset_index(drop=True)
        data_right=data_right.reset_index(drop=True)
        
        if data_left.shape[0]==0 or data_right.shape[0]==0:
            if x_train.Survived.mean()>0.5:
                self.target="Survived"
            else:
                self.target="Dead"
            return  
        
        if(self.depth>=self.max_depth):
            if x_train.Survived.mean()>0.5:
                self.target="Survived"
            else:
                self.target="Dead"
            return  
        self.left=DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(data_left)
        
        self.right=DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(data_right)
        
        if(x_train.Survived.mean()>=0.5):
            self.target="Survived"
        else:
            self.target="Dead"
        return
    def predict(self,test):
        if test[self.fkey]>self.fval:
            #go to right
            if self.right is None:
                    return self.target
            
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)
        

In [297]:
split=int(0.7*data_clean.shape[0])
train_data=data_clean[:split][:]
test_data=data_clean[split:][:]
test_data=test_data.reset_index(drop=True)


In [299]:
dt=DecisionTree()

In [ ]:
dt.train(train_data)

In [364]:
y_pred=[]
for i in range(test_data.shape[0]):
    y_pred.append(dt.predict(test_data.loc[i]))
print(y_pred,test_data["Survived"])

['Survived', 'Survived', 'Survived', 'Dead', 'Dead', 'Survived', 'Survived', 'Survived', 'Survived', 'Survived', 'Dead', 'Dead', 'Dead', 'Survived', 'Survived', 'Dead', 'Survived', 'Survived', 'Dead', 'Dead', 'Survived', 'Dead', 'Survived', 'Survived', 'Survived', 'Survived', 'Dead', 'Survived', 'Dead', 'Survived', 'Dead', 'Dead', 'Survived', 'Survived', 'Dead', 'Dead', 'Dead', 'Survived', 'Survived', 'Survived', 'Survived', 'Survived', 'Survived', 'Dead', 'Survived', 'Survived', 'Dead', 'Dead', 'Survived', 'Dead', 'Dead', 'Dead', 'Survived', 'Survived', 'Dead', 'Dead', 'Dead', 'Dead', 'Survived', 'Survived', 'Survived', 'Survived', 'Survived', 'Survived', 'Survived', 'Survived', 'Dead', 'Survived', 'Dead', 'Survived', 'Survived', 'Survived', 'Dead', 'Survived', 'Dead', 'Survived', 'Survived', 'Dead', 'Survived', 'Dead', 'Survived', 'Survived', 'Survived', 'Survived', 'Survived', 'Dead', 'Survived', 'Dead', 'Survived', 'Survived', 'Survived', 'Dead', 'Survived', 'Dead', 'Dead', 'Surviv

In [403]:
def encode(data):
    if data=="Survived":
        return 0
    else:
        return 1

In [404]:
y_pred=np.array(y_pred).reshape((-1,1))
y=[]
for i in range(y_pred.shape[0]):
    y.append(encode(y_pred[i]))

In [405]:
y_predict=np.array(y).reshape((-1,1))
y_actual=np.array(test_data["Survived"]).reshape((-1,1))

In [406]:
accuracy=np.sum(y_predict==y_actual)/y_pred.shape[0]

In [407]:
print(accuracy)

0.6716417910447762
